# Calculate Technical Indicators for TSLA
We will calculate the 2 Exponential Moving Averages (EMA)s required to calculate Moving Average Convergence Divergence (MACD); MACD; and Relative Strength Index (RSI)

**The Exponential Moving Average (EMA)** is a moving average that places a greater weight and significance on the most recent data points.
https://www.investopedia.com/terms/e/ema.asp

$$EMA_{today} = \left(Value_t * ( \frac{Smoothing}{1 + Days})\right) + EMA_{yesterday} * \left(1-( \frac{Smoothing}{1 + Days})\right)$$

It can be calculated directly by pandas.


**The Moving Average Convergance Divergance (MACD) indicator** is calculated by subtracting the 26-period EMA from the 12-period EMA.
https://www.investopedia.com/terms/m/macd.asp

$$MCAD = {12 period EMA - 26 period EMA}$$

**The Relative Strength Index (RSI)** measures the size of price changes. This is an indication that the stock is overbought or oversold. An RSI > 70 is typically deemed to be overbought and a RSI < 30 is typically deemed to be oversold.

* Overbought: When a security is believed to be trading at a level above its intrinsic or fair value
* Oversold: When a security is thought to be trading below its intrinsic value

https://www.investopedia.com/terms/r/rsi.asp

It can be calculated by pandas-ta using ta-lib

## Get Daily TSLA Candles
We will need 3 months worth so will need to retrieve:
* 92 days; 
* plus an extra 26 days for the first EMA 26 period moving average calculations;
* plus and extra 9 days for the first MACD EMA 9 period calculations.

In [1]:
from pandas_datareader import data as dr
import datetime as dt

 # We will calculate for the last 3 months so will need 3 months worth of data + and extra 26 days for the first EMA
# calculations and an exita 9 days for the first MACD EMA9 calculation which will be used for the signal line
from_date = dt.datetime.now() - dt.timedelta(days=92 + 26 + 9)
candles = dr.DataReader('TSLA', data_source='yahoo', start=from_date)

# Sort
candles = candles.sort_index()

candles

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-11-23,1180.500000,1062.699951,1167.510010,1109.030029,36171700,1109.030029
2021-11-24,1132.770020,1062.000000,1080.390015,1116.000000,22560200,1116.000000
2021-11-26,1108.780029,1081.000000,1099.469971,1081.920044,11680900,1081.920044
2021-11-29,1142.670044,1100.189941,1100.989990,1136.989990,19464500,1136.989990
2021-11-30,1168.000000,1118.000000,1144.369995,1144.760010,27092000,1144.760010
...,...,...,...,...,...,...
2022-03-24,1024.489990,988.799988,1009.729980,1013.919983,22973600,1013.919983
2022-03-25,1021.799988,997.320007,1008.000000,1010.640015,20642900,1010.640015
2022-03-28,1097.880005,1053.599976,1065.099976,1091.839966,34168700,1091.839966


## MACD; MACD Signal; MACD - MACD Signal; and RSI
Use close price. For each row in candle data calculate:
* MACD, which will require:
    * 26 period EMA
    * 12 period EMA
* MACD Signal, which will require:
    * MACD
    * 9 period EMA of MACD
* MACD - MACD Signal
* RSI

In [2]:
import pandas_ta as ta

# Creare a copy so we dont change the original candle data
data = candles.copy()

# Add 26 period EMA, 12 period EMA and MACD
data['EMA26'] = data['Close'].ewm(span=26, min_periods=26, adjust=False).mean()
data['EMA12'] = data['Close'].ewm(span=12, min_periods=12, adjust=False).mean()
data['MACD'] = (data["EMA12"] - data["EMA26"])

# Add 9 period EMA of MACD and MACD Signal
data['MACD_SIG'] = data['MACD'].ewm(span=9, min_periods=9, adjust=False).mean()
data['MACD_vs_SIG'] = (data["MACD"] - data["MACD_SIG"])

# Add RSI
data['RSI'] = data.ta.rsi(close=data['Close'])

# Remove first 35 rows, 26 of these were only used to calculate the first moving averages and 9 of these were only used 
# to calculate the signal line.
data = data.iloc[35: , :].copy()

data

,High,Low,Open,Close,Volume,Adj Close,EMA26,EMA12,MACD,MACD_SIG,MACD_vs_SIG,RSI
Date,,,,,,,,,,,,
2022-01-13,1115.599976,1026.540039,1109.069946,1031.560059,32403300,1031.560059,1060.900534,1066.746205,5.845672,8.064434,-2.218762,46.339958
2022-01-14,1052.000000,1013.380005,1019.880005,1049.609985,24308100,1049.609985,1060.064197,1064.109864,4.045667,7.260681,-3.215014,48.208533
2022-01-18,1070.790039,1016.059998,1026.609985,1030.510010,22247800,1030.510010,1057.874998,1058.940655,1.065658,6.021676,-4.956018,46.368504
2022-01-19,1054.670044,995.000000,1041.709961,995.650024,25147500,995.650024,1053.265740,1049.203635,-4.062105,4.004920,-8.067025,43.132671
2022-01-20,1041.660034,994.000000,1009.729980,996.270020,23496200,996.270020,1049.043835,1041.060002,-7.983833,1.607169,-9.591002,43.208580
2022-01-21,1004.549988,940.500000,996.340027,943.900024,34472000,943.900024,1041.255405,1026.112313,-15.143092,-1.742883,-13.400209,38.530071
2022-01-24,933.510010,851.469971,904.760010,930.000000,50521900,930.000000,1033.014264,1011.325803,-21.688460,-5.731998,-15.956462,37.373380
2022-01-25,951.260010,903.210022,914.200012,918.400024,28865300,918.400024,1024.524320,997.029530,-27.494790,-10.084557,-17.410233,36.391532
2022-01-26,987.690002,906.000000,952.429993,937.409973,34955800,937.409973,1018.071405,987.857290,-30.214115,-14.110468,-16.103647,39.210059


## Plot the data
We will plot three charts:
* Candles with the EMA26 and EMA12;
* MACD vs signal line (MACD EMA 9);
* RSI; and
* Volume.

### Chart setup
* Declare imports;
* Define hovertool;
* Define candle and bar widths;
* Define date format; and
* Define function to format plots.

In [4]:
from bokeh import layouts as bl
from bokeh import models as bm
from bokeh import plotting as bp
from bokeh import io as io

io.output_notebook()

# Set the datasource for the chart
source = bm.ColumnDataSource(data)

# Date formatter. Used on axis
dtfmt = bm.DatetimeTickFormatter(days='%Y-%m-%d')

# Display date and OHLCV values for hover
hover = bm.HoverTool(
    tooltips=[
        ("Date", "@Date{'%Y-%m-%d'}"),
        ("Open", "@Open{'0,000.00'}"),
        ("High", "@High{'0,000.00'}"),
        ("Low", "@Low{'0,000.00'}"),
        ("Close", "@Close{'0,000.00'}"),
        ("Volume", "@Volume{'0,000'}"),
        ("26 Period EMA", "@EMA26{'0,000.00'}"),
        ("12 Period EMA", "@EMA12{'0,000.00'}"),
        ("MACD", "@MACD{'0,000.00'}"),
        ("Signal", "@MACD_SIG{'0,000.00'}"),
        ("RSI", "@RSI{'0,000.00'}"),
    ],

    formatters={
        '@Date': 'datetime'
    },
    mode='mouse'
)

# Bar width for candles and bar charts will be the number of milliseconds between the times - 20% for spacing.
bar_width = (data.index[1] - data.index[0]).total_seconds() * 1000 * .8

# Function to format plots
def format_plot(p):
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "7px"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 1.0
    p.grid.grid_line_alpha = 0.3
    p.xaxis[0].formatter = dtfmt

Loading BokehJS ...

### Plot the candles with both EMAs

In [5]:
p1 = bp.figure(plot_width=1000, x_axis_type="datetime", tools=[hover], x_axis_location="below", plot_height=300)

# Candle for candle chart colour will depend on whether it has increased or decreased between open and close
inc = bm.CDSView(source=source, filters=[bm.BooleanFilter(source.data['Close'] > source.data['Open'])])
dec = bm.CDSView(source=source, filters=[bm.BooleanFilter(source.data['Open'] > source.data['Close'])])
views = {inc: '#555555', dec: '#F2583E'}  # Colours for inc and dec views

# Draw the Candle wicks
p1.segment(source=source, x0='Date', y0='High', x1='Date', y1='Low', color="black")

# Draw the candle body. Colour will depend on open / close prices and will use inc and dec views declared above.
for view in views:
    p1.vbar(source=source, view=view, x='Date', top='Open', bottom='Close', width=bar_width,
            fill_color=views[view], line_color="black")
    
# Plot the EMA lines
p1.line(source=source, x='Date', y='EMA12', color="green", legend_label = '12 Period EMA')
p1.line(source=source, x='Date', y='EMA26', color="lightblue", legend_label = '26 Period EMA')

# Format the plot
format_plot(p1)

# Show
bp.show(p1)

### Plot the MACD vs its signal line
Signal line is the 9 period EMA of the MACD

Histogram will plot the difference between the MACD and its signal line

In [6]:
p2 = bp.figure(plot_width=1000, x_axis_type="datetime", tools=[hover], plot_height=300)

# Plot the baseline and MACD lines
p2.line(source=source, x='Date', y='MACD', color="green", legend_label = 'MACD')
p2.line(source=source, x='Date', y='MACD_SIG', color="lightblue", legend_label = 'Signal')

# Plot the histogram. Colour will depend on whether it is positive or negative
inc = bm.CDSView(source=source, filters=[bm.BooleanFilter(source.data['MACD_vs_SIG'] >= 0)])
dec = bm.CDSView(source=source, filters=[bm.BooleanFilter(source.data['MACD_vs_SIG'] < 0)])
views = {inc: '#555555', dec: '#F2583E'}  # Colours for inc and dec views

for view in views:
    p2.vbar(source=source, view=view, x='Date', top='MACD_vs_SIG', width=bar_width, fill_color=views[view], 
            line_color="black")
                                         
# Format the plot
format_plot(p2)

# Show
bp.show(p2)

### Plot RSI

In [7]:
p3 = bp.figure(plot_width=1000, x_axis_type="datetime", tools=[hover], plot_height=200)

# RSI Line
p3.line(source=source, x='Date', y='RSI', color="green", legend_label = 'RSI')

# Overbought and oversold lines
p3.line(source=source, x='Date', y=70, color='red', legend_label = 'Overbought > 70')
p3.line(source=source, x='Date', y=30, color='black', legend_label = 'Oversold < 30')

# Format the plot
format_plot(p3)

# Show
bp.show(p3)

### Plot the volume

In [8]:
p4 = bp.figure(plot_width=1000, x_axis_type="datetime", tools=[hover], plot_height=200)

# Volume bars
p4.vbar(source=source, x='Date', top='Volume', width=bar_width)

# Format the plot
format_plot(p4)

# Show
bp.show(p4)